In [22]:
import cobra #as cb
import pandas as pd
#import tabulate
import re
import sys
import getopt
import os.path
import copy
import csv
import math
import cobra.flux_analysis.variability
#import massedit
import subprocess
import shutil, errno
import statistics
from cobra import Reaction
import comets as c

"""def comets(layout_file, params_file=None):

#establish layout with a file with string of the model's name
    layout=c.layout(layout_file)
    if params_file != None:
        #establish params with a global params file
        params = c.params(global_params = params_file)
    else:
        #default params
        params = c.params()
        
    return layout, params"""

def Flycop(layout_file, fitFunc, dirPlot='', repeat=10, met1={}, met2={}, strains=[], params_file=None, uptake=False):
    """For now, using bash and R script, just two metabolites
       Input: layout_file in comets format with the model/s in it.
       Fitfunc: function for joint fitness. Select one of the following:
           -'Yield'
           -'MaxYield_MinTime'
           -'YieldNewScattered'
           -'MaxYieldNewScattered_MinTime'
           -'Biomass'
           -'MaxBiomass_MinTime'
           -'MaxGR_MinTime'
       Met1: dictionary for metabolite 1 with the following keys: 'name', 'MW','ID' (as o2_e) ...(need more?)
       """
    if not(os.path.exists('IndividualRunsResults')):
        os.makedirs('IndividualRunsResults')
    totfitness=0
    sumTotBiomass=0
    sumTotYield=0
    fitnessList=[]
   #establish layout with a file with string of the model's name
    layout=c.layout(layout_file)
    if params_file != None:
        #establish params with a global params file
        params = c.params(global_params = params_file)
    else:
        #default params
        params = c.params()
    #establish comets class
    
    comets = c.comets(layout, params)
# To repeat X times, due to random behaviour in COMETS:
    for i in range(repeat):
        with open("output.txt", "w") as f:  #-----------don't know if needed
            comets.run(delete_files=False)
        
        #graphic part
        subprocess.call(['../../Scripts/plot_biomassX2_vs_2mediaItem.sh','template', met1['nombre'], met2['nombre'], \
                         strains[0], strains[1]])
        #pending further generalization
        
        # 7.- Compute fitness (measure to optimize):
        print('computing fitness...')
        # 7.1.- Determine endCycle: when glucose and acetate are exhausted
        with open("biomass_vs_"+met1['nombre']+"_"+ met2['nombre']+"_template.txt", "r") as sources:
            lines = sources.readlines()
            iniPointV=lines[0].split()
            iniBiomass=float(iniPointV[1])+float(iniPointV[2])
            totmet1=float(iniPointV[3])
            endmet1Cycle=0
            for line in lines:
                endCycle=int(line.split()[0])
                met1Conc=float(line.split()[3])
                met2Conc=float(line.split()[4])
                if((endmet1Cycle==0)and(met1Conc==0.0)):
                    endGlcCycle=endCycle
                if((met1Conc==0.0)and(met2Conc==0.0)):
                    break;
                if((met1Conc==0.0)and(ac1>=0)and(ac2>=0)): #don't know how to approach this
                    #making it an arg? still too specific
                    #another end condition?
                    break;
            endPointV=lines[endCycle].split()
         # 7.2.- Compute first element fitness: maximize biomass yield
        # To compute final biomass as the maximum biomass of each strain
            finalBiomass1=0
            finalBiomass2=0
            count=0
            for line in lines:
                if(float(line.split()[1])>finalBiomass1):
                    finalBiomass1=float(line.split()[1])
                if(float(line.split()[2])>finalBiomass2):
                    finalBiomass2=float(line.split()[2])
                if(count>endCycle):
                    break;
                count=count+1
            finalBiomass=finalBiomass1+finalBiomass2
            biomassYieldNew=float((finalBiomass-iniBiomass)/(totmet1*met1['WH'])) # molecular weigth of met1 per mmol
            # For normalizing yield
            MaximumYield=0.6
            # 7.3.- Compute second element fitnes: minimize time        
            fitTime=1-(float(endCycle)/float(240))
            # 7.4.- Compute joint fitness, as a 50% each element.
            if(fitFunc=='Yield'):
                fitness=(biomassYieldNew/MaximumYield) # Normalizing Yield
            elif(fitFunc=='MaxYield_MinTime'):
                fitness=0.5*(biomassYieldNew/MaximumYield)+0.5*fitTime #Normalizing yield
            elif(fitFunc=='YieldNewScattered'): # (biomass^4)*10: To spread values from ~0.45-0.55 values to 0.5 to 1
                fitness=(biomassYieldNew**4)*10
            elif(fitFunc=='MaxYieldNewScattered_MinTime'):
                fitness=0.5*((biomassYieldNew**4)*10)+0.5*fitTime
            elif(fitFunc=='Biomass'):
                fitness=float(finalBiomass-iniBiomass)
            elif(fitFunc=='MaxBiomass_MinTime'):
                fitness=0.5*(float(finalBiomass-iniBiomass))+0.5*fitTime
                
                #maybe this can be done without bash?
            elif((fitFunc=='GR')or(fitFunc=='MaxGR_MinTime')):
                numRxnGR1=int(subprocess.check_output(['egrep -A1 "OBJECTIVE"' +layout.models[0]+ '| tail -1 | tr -d \[:space:\]'], \
                                                      shell=True)) #Works this way??????
                numRxnGR2=int(subprocess.check_output(['egrep -A1 "OBJECTIVE"' +layout.models[1]+ '| tail -1 | tr -d \[:space:\]'], \
                                                  shell=True)) 
            try:
                GR1=float(subprocess.check_output(['egrep "fluxes\{"'+str(endmet1Cycle-1)+'"\}\{1\}\{1\}\{1\}" flux_log_template.txt |  cut -d"=" -f2 | cut -d" " -f'+str(numRxnGR1+1)], shell=True))
            except:
                GR1=0.0
            try:
                GR2=float(subprocess.check_output(['egrep "fluxes\{"'+str(endmet1Cycle-1)+'"\}\{1\}\{1\}\{2\}" flux_log_template.txt | cut -d"=" -f2 | cut -d" " -f'+str(numRxnGR2+1)], shell=True))
            except:
                GR2=0.0
                fitGR=(GR1+GR2)/2
                if(fitFunc=='GR'):
                    fitness=fitGR
                elif(fitFunc=='MaxGR_MinTime'):
                    fitness=0.5*fitGR+0.5*fitTime
            if(float(finalBiomass-iniBiomass) > 1.03):  # Given that with both strains with WT, total biomass=1.028
            fitness=0
            
            if uptake = True:
                totalUptake = calculate_uptake(met2['ID'], strains)
            
            print(" Total biomass: "+str(round(finalBiomass,6))+" in cycle "+str(endCycle)+". Biomass yield="+str(round(biomassYieldNew,6)))
   
            totfitness=totfitness+fitness
            fitnessList.append(fitness)
            sumTotBiomass=sumTotBiomass+finalBiomass
            sumTotYield=sumTotYield+biomassYieldNew
            
            # Copy individual solution
            file='IndividualRunsResults/'+'biomass_vs_'+met1['nombre']+'_'+ met2['nombre']+'_run'+str(i)+'_'+str(fitness)+'_'+str(endCycle)+'.pdf'        
            shutil.move('biomass_vs_glc_D_ac_template_plot.pdf',file)
            if(dirPlot != ''):
                #Don't know how to deal with that
                #file2=dirPlot+'biomass_vs_glc_D_ac_'+str(glu1)+'_'+str(ac1)+'_'+str(o21)+'_'+str(glu2)+'_'+str(ac2)+'_'+str(o22)+'_'+str(round(uptakeAc1,1))+'_'+str(round(uptakeAc2,1))+'_run'+str(i)+'_'+str(fitness)+'_'+str(endCycle)+'.pdf'
                #shutil.copy(file,file2)
            file='IndividualRunsResults/'+'total_biomass_log_run'+str(i)+'.txt'
            shutil.move('total_biomass_log_template.txt',file)
            file='IndividualRunsResults/'+'media_log_run'+str(i)+'.txt'
            shutil.move('media_log_template.txt',file)
            file='IndividualRunsResults/'+'flux_log_run'+str(i)+'.txt'
            shutil.move('flux_log_template.txt',file) 
            
    avgfitness=totfitness/repeat
    sdfitness=statistics.stdev(fitnessList)
    avgBiomass=sumTotBiomass/repeat
    avgYield=sumTotYield/repeat
    
    print("Fitness_function\tconfiguration\tfitness\tsd\tavg.Biomass\tavg.Yield\tendCycle")
  #again
    #print(fitFunc+"\t"+str(glu1)+','+str(ac1)+','+str(o21)+','+str(glu2)+','+str(ac2)+','+str(o22)+','+str(round(uptakeAc1,1))+','+str(round(uptakeAc2,1))+"\t"+str(round(avgfitness,6))+"\t"+str(sdfitness)+"\t"+str(round(avgBiomass,6))+"\t"+str(round(avgYield,6))+"\t"+str(endCycle))
    with open(dirPlot+"configurationsResults"+fitFunc+".txt", "a") as myfile:
        myfile.write("Fitness_function\tconfiguration\tfitness\tsd\tavg.Biomass\tavg.Yield\tendCycle\n")
       # myfile.write(fitFunc+"\t"+str(glu1)+','+str(ac1)+','+str(o21)+','+str(glu2)+','+str(ac2)+','+str(o22)+','+str(round(uptakeAc1,1))+','+str(round(uptakeAc2,1))+"\t"+str(round(avgfitness,6))+"\t"+str(sdfitness)+"\t"+str(round(avgBiomass,6))+"\t"+str(round(avgYield,6))+"\t"+str(endCycle)+"\n")
  
    print("Avg.fitness(sd):\t"+str(avgfitness)+"\t"+str(sdfitness)+"\n")
    if(sdfitness>0.1):
        avgfitness=0.0
  
  return avgfitness,sdfitness

    def calculate_uptake(met_id, strains):
        #Compute metabolite uptake
        model=layout.models[0]
        df = model.smat
        numRxnMet= df[df['metabolite'] == met_id].index # Position of metabolite in model
        #Is this alright? Just a single row for met? Possible 2 or more reactions per metabolite?
        
        
        # flux in cycle 2 (the first one is usually 0) #maybe this can be done without bash?
        totalUptake=[]
        #meanwhile, adapting it to be for n models
        for i in len(layout.models):
        uptakeMet=float(subprocess.check_output(['egrep "fluxes\{2\}\{1\}\{1\}\{'+i+'\}" flux_log_template.txt | cut -d"=" -f2 | cut -d" " -f'+str(numRxnMet)], shell=True))
        totalUptake.append(uptakeMet)
        return totalUptake
    
    """def initialize_models():
 # Only to run 1st time, to build the models!!
 if not(os.path.exists('ModelsInput/iJO1366.mat')):
     print('ERROR! Not iJO1366.mat files with GEM of consortium strains in ModelsInput!')
 else:
  path=os.getcwd()
  os.chdir('ModelsInput')
  model=cobra.io.load_matlab_model('iJO1366.mat')
  # To change the core to the WT biomass as objective
  model.objective="BIOMASS_Ec_iJO1366_WT_53p95M"
  # To remove one copy from GLCtex
  model.reactions.get_by_id('GLCtex_copy2').bounds=(0,0)
  model.reactions.get_by_id('PFK_3').bounds=(0,0) # To avoid to deviate flux of glycolysis, going it through PFK rxn.
  # Replace brackets with compartment location (e.g. "[c]") in metabolite ids by '_' (e.g. "_c") 
  for metabolite in model.metabolites:
    metabolite.id = re.sub('_c$',r'[c]',metabolite.id)
    metabolite.id = re.sub('_p$',r'[p]',metabolite.id)
    metabolite.id = re.sub('_e$',r'[e]',metabolite.id)
    metabolite.id = re.sub('__',r'_',metabolite.id)
    metabolite.compartment = ''
  # To solve possible problems in changing names     
  model.repair()
  # Replace brackets with compartment location (e.g. "[c]") in metabolite ids by '_' (e.g. "_c") 
  for rxn in model.reactions:
    rxn.id = re.sub('_p$',r'(p)',rxn.id)
    rxn.id = re.sub('_c$',r'(c)',rxn.id)
    rxn.id = re.sub('_e$',r'(e)',rxn.id)    
  # To solve possible problems in changing names     
  model.repair()
  cobra.io.save_matlab_model(model,"iJO1366py.mat",'model')
  #
  model.reactions.get_by_id('EX_o2(e)').bounds=(-14.4,1000) # To adjust WT model to realistic values of acetate generation, according to [Steinsiek,2012; https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3486086/], with: glc~=10, ac~=3.7, GR~=0.8
  model.reactions.get_by_id('EX_ac(e)').bounds=(-1000,60) # Open lower limit of acetate (allowing to taking it (for cases where there isn't glucose), and limiting the secretion to 3*maximum glucose in our long term experiment (20))
  cobra.io.save_matlab_model(model,"iJO1366py_tmp.mat",'model')
  del(model)
  os.chdir(path)
# end-def"""
    """################################################################    
### FUNCTION mat_to_comets #####################################    
# mat_to_comets(modelPath)
# Re-code in python from COMETS code
def mat_to_comets(matInputFile):
    model=cobra.io.load_matlab_model(matInputFile)
    # Open output file:
    with open(matInputFile+'.cmt', mode='w') as f:
        # Print the S matrix
        f.write("SMATRIX  "+str(len(model.metabolites))+"  "+str(len(model.reactions))+"\n")
        for x in range(len(model.metabolites)):
            for y in range(len(model.reactions)):
                if (model.metabolites[x] in model.reactions[y].metabolites):
                    coeff=model.reactions[y].get_coefficient(model.metabolites[x])
                    f.write("    "+str(x+1)+"   "+str(y+1)+"   "+str(coeff)+"\n")
        f.write("//\n")
        
        # Print the bounds
        f.write("BOUNDS  -1000  1000\n");
        for y in range(len(model.reactions)):
            lb=model.reactions[y].lower_bound
            up=model.reactions[y].upper_bound
            f.write("    "+str(y+1)+"   "+str(lb)+"   "+str(up)+"\n")
        f.write("//\n")
        
        # Print the objective reaction
        f.write('OBJECTIVE\n')
        for y in range(len(model.reactions)):
            if (model.reactions[y] in model.objective):
                indexObj=y+1
        f.write("    "+str(indexObj)+"\n")
        f.write("//\n")
        
        # Print metabolite names
        f.write("METABOLITE_NAMES\n")
        for x in range(len(model.metabolites)):
            f.write("    "+model.metabolites[x].id+"\n")
        f.write("//\n")

        # Print reaction names
        f.write("REACTION_NAMES\n")
        for y in range(len(model.reactions)):
            f.write("    "+model.reactions[y].id+"\n")
        f.write("//\n")

        # Print exchange reactions
        f.write("EXCHANGE_REACTIONS\n")
        for y in range(len(model.reactions)):
            if (model.reactions[y].id.find('EX_')==0):
                f.write(" "+str(y+1))
        f.write("\n//\n")                
### end-function-mat_to_comets    
################################################################
"""### FUNCTION ecoliLongTermFLYCOP_oneConf #######################   
def ecoliLongTermFLYCOP_oneConf(glu1,ac1,o21,glu2,ac2,o22,fitFunc='MaxYield_MinTime',dirPlot='',repeat=10):
  '''
  Call: avgFitness, sdFitness = ecoliLongTerm_oneConf(glu1,ac1,o21,glu2,ac2,o22)

  INPUTS: glu1: lower bound of glucose uptake in model 1.
          ace1: lower bound of acetate uptake in model 1.
          o21: lower bound of oxygen uptake in model 1.
          glu2: lower bound of glucose uptake in model 2.
          ace2: lower bound of acetate uptake in model 2.
          o22: lower bound of oxygen uptake in model 2.
          dirPlot: copy of the graphs with several run results.
          repeat: number of runs with the same configuration.
  OUTPUT: avgFitness: average fitness of 'repeat' COMETS runs with the same configuration (due to it is not deterministic)
          sdFitness: standard deviation of fitness during 'repeat' COMETS runs (see above)
  '''

  """if not(os.path.exists('ModelsInput/iJO1366py_tmp.mat')):
      initialize_models()

  # Determine initial biomasses.
  biomass1=0.01
  biomass2=0.01

  print("Fitness function:"+fitFunc)

  # Single GEMs parameter modifications
  # ===================================
    if not(os.path.exists('ecoli_1_tmp.mat.cmt')):
    # 1.1.- [COBRApy] Establish modifications in model 1 
    model1=cobra.io.load_matlab_model('ModelsInput/iJO1366py_tmp.mat')
    model1.reactions.get_by_id('EX_glc__D(e)').bounds=(-1000,1000)
    model1.reactions.get_by_id('EX_o2(e)').bounds=(-1000,1000)
    model1.reactions.get_by_id('GLCtex_copy1').bounds=(0,-glu1)
    model1.reactions.get_by_id('O2tex').bounds=(0,-o21)
    if(ac1<=0):
        model1.reactions.get_by_id('ACtex').lower_bound=-1000
        model1.reactions.get_by_id('ACtex').upper_bound=-ac1
        model1.reactions.get_by_id('EX_ac(e)').bounds=(ac1,1000)
    else:
        model1.reactions.get_by_id('ACtex').lower_bound=-ac1
        model1.reactions.get_by_id('ACtex').upper_bound=1000
        model1.reactions.get_by_id('EX_ac(e)').bounds=(-1000,ac1)
    model1.initial_pop=biomass1
    cobra.io.save_matlab_model(model1,'ecoli_1_tmp.mat','model')
    
      
    # 1.2.- [COBRApy] Establish modifications in model 2
    model2=cobra.io.load_matlab_model('ModelsInput/iJO1366py_tmp.mat')
    model2.reactions.get_by_id('EX_glc__D(e)').bounds=(-1000,1000)
    model2.reactions.get_by_id('EX_o2(e)').bounds=(-1000,1000)
    model2.reactions.get_by_id('GLCtex_copy1').bounds=(0,-glu2)
    model2.reactions.get_by_id('O2tex').bounds=(0,-o22)
    if(ac2<=0):
        model2.reactions.get_by_id('ACtex').lower_bound=-1000
        model2.reactions.get_by_id('ACtex').upper_bound=-ac2
        model2.reactions.get_by_id('EX_ac(e)').bounds=(ac2,1000)
    else:
        model2.reactions.get_by_id('ACtex').lower_bound=-ac2
        model2.reactions.get_by_id('ACtex').upper_bound=1000
        model2.reactions.get_by_id('EX_ac(e)').bounds=(-1000,ac2)
        model2.initial_pop=biomass2

    cobra.io.save_matlab_model(model2,'ecoli_2_tmp.mat','model')
    del(model)
    
    # 2.- [python] 
    mat_to_comets('ecoli_1_tmp.mat')
    mat_to_comets('ecoli_2_tmp.mat')

    # Community parameter modifications
    # =================================        
    # 4.- [shell script] Write automatically the COMETS parameter about initial biomass of strains.
    massedit.edit_files(['ecoliLongTerm_layout_template.txt'],["re.sub(r'XXX','"+str(biomass1)+"',line)"], dry_run=False)
    massedit.edit_files(['ecoliLongTerm_layout_template.txt'],["re.sub(r'YYY','"+str(biomass2)+"',line)"], dry_run=False)        
"""